# Content-based Recommendation system

## Simple Content-based Filtering 

In [1]:
# import library
import pandas as pd

# 1. Membaca dataset
hotel_df = pd.read_excel('data_hotel.xlsx') 
hotel_df.head()

,hotel_id,hotel_name,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night
0,h0001,Midtown Residence Marvell City Surabaya,NaN,Jawa Timur,Surabaya,Jalan Ngagel Raya No 123,527866.666667
1,h0002,favehotel Graha Agung Surabaya,Sebuah Hotel Mewah di Surabaya Persembahan dar...,Jawa Timur,Surabaya,Jl. Mayjen Yono Soewoyo Pakuwon Indah Square A...,442860.000000
2,h0003,The Sun Hotel Sidoarjo,Hotel Bintang 3 Pertama dan Satu-satunya di Si...,Jawa Timur,Surabaya,"Jl. Pahlawan No.1,Sidokumpul, Sidoarjo, Suraba...",305000.000000
3,h0004,Grand Surabaya Hotel,Penginapan Yang Tenang Dan Nyaman Di Surabaya.,Jawa Timur,Surabaya,"Jl. Pemuda 19-21, Surabaya, Indonesia",324999.333333
4,h0005,The WIN Hotel Surabaya,WIN Hotel adalah hotel smart bintang 3 yang me...,Jawa Timur,Surabaya,"Jl. Embong Tanjung 46 - 48 Surabaya, Jawa Timu...",310947.250000


In [2]:
# 2. Data cleaning
# Cek kolom yang terdapat null
hotel_df.isnull().sum()

hotel_id               0
hotel_name             0
hotel_description    101
hotel_province         0
hotel_city             0
hotel_address          0
price_per_night        6
dtype: int64

In [3]:
# Gunakan data hanya yang hotel description tidak null
hotel_df = hotel_df[hotel_df['hotel_description'].notnull()]

In [4]:
# 3. Text Processing
# Impport library TFIDF dan Sastrawi
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import re
import random

In [5]:
# Gunakan library Sastrawi untuk stopword bahasa indonesia
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
sastrawi = StopWordRemoverFactory()
stopworda = sastrawi.get_stop_words()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = stemmer.stem(text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

In [6]:
# Proses steamming dan remove stopword  ini perlu proses lama
hotel_df['desc_clean'] = hotel_df['hotel_description'].apply(clean_text)
hotel_df.head()

,hotel_id,hotel_name,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night,desc_clean
1,h0002,favehotel Graha Agung Surabaya,Sebuah Hotel Mewah di Surabaya Persembahan dar...,Jawa Timur,Surabaya,Jl. Mayjen Yono Soewoyo Pakuwon Indah Square A...,442860.000000,buah hotel mewah surabaya sembah archipelago i...
2,h0003,The Sun Hotel Sidoarjo,Hotel Bintang 3 Pertama dan Satu-satunya di Si...,Jawa Timur,Surabaya,"Jl. Pahlawan No.1,Sidokumpul, Sidoarjo, Suraba...",305000.000000,hotel bintang 3 pertama satusatunya sidoarjo
3,h0004,Grand Surabaya Hotel,Penginapan Yang Tenang Dan Nyaman Di Surabaya.,Jawa Timur,Surabaya,"Jl. Pemuda 19-21, Surabaya, Indonesia",324999.333333,inap tenang nyaman surabaya
4,h0005,The WIN Hotel Surabaya,WIN Hotel adalah hotel smart bintang 3 yang me...,Jawa Timur,Surabaya,"Jl. Embong Tanjung 46 - 48 Surabaya, Jawa Timu...",310947.250000,win hotel hotel smart bintang 3 tawar layan tu...
5,h0006,POP! Hotel Stasiun Kota Surabaya,Sebuah Hotel Modern yang Terjangkau di Jawa Timur,Jawa Timur,Surabaya,"Jl. Waspada No.58, Surabaya, Jawa Timur, Indon...",276333.333333,buah hotel modern jangkau jawa timur


In [7]:
# 4. Hitung TF-IDF dan Cosine similarity
hotel_df.set_index('hotel_name', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,)
tfidf_matrix = tf.fit_transform(hotel_df['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.00399128, 0.05252692, ..., 0.00584594, 0.00556788,
        0.0025404 ],
       [0.00399128, 1.        , 0.        , ..., 0.00263359, 0.00250833,
        0.00328081],
       [0.05252692, 0.        , 1.        , ..., 0.00533054, 0.005077  ,
        0.0536853 ],
       ...,
       [0.00584594, 0.00263359, 0.00533054, ..., 1.        , 0.6430531 ,
        0.01276909],
       [0.00556788, 0.00250833, 0.005077  , ..., 0.6430531 , 1.        ,
        0.01216174],
       [0.0025404 , 0.00328081, 0.0536853 , ..., 0.01276909, 0.01216174,
        1.        ]])

In [8]:
indices = pd.Series(hotel_df.index)
indices[:15]

0       favehotel Graha Agung Surabaya
1               The Sun Hotel Sidoarjo
2                 Grand Surabaya Hotel
3               The WIN Hotel Surabaya
4     POP! Hotel Stasiun Kota Surabaya
5                 Evora Hotel Surabaya
6         MaxOneHotels at Dharmahusada
7              Zoom Dharmahusada Hotel
8          Neo+ Waru Sidoarjo by ASTON
9                   favehotel Sidoarjo
10       News Front One Hotel Surabaya
11             Gunawangsa Manyar Hotel
12       Zest Hotel Jemursari Surabaya
13             The Life Hotel Surabaya
14          Best Western Papilio Hotel
Name: hotel_name, dtype: object

In [9]:
# 5. Modelling recommender system
def recommendations(name, top = 10):
    
    recommended_hotel = []
    
    idx = indices[indices == name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    top = top + 1
    top_indexes = list(score_series.iloc[0:top].index)
    
    for i in top_indexes:
        recommended_hotel.append(list(hotel_df.index)[i]+" - "+str(score_series[i]))
        
    return recommended_hotel

In [10]:
# Contoh menampilkan rekomendasi hotel
recommendations('Gunawangsa Manyar Hotel', 15)

['Gunawangsa Manyar Hotel - 1.0000000000000002',
 'Midtown Hotel Surabaya - 0.17992312860891296',
 'Aria Gajayana Hotel - 0.14461427726403756',
 'GP Mega Kuningan Hotel - 0.12367390420720603',
 'Le Polonia Medan managed by Topotels - 0.12049561758006845',
 'Swiss-Belinn Manyar Surabaya - 0.11731929279752672',
 'Hotel Santika Premiere Dyandra Hotel & Convention - 0.11442553936688896',
 'Best Western Plus Kemayoran Hotel - 0.10975989716138217',
 'Swiss-Belinn Malang - 0.09085670933358225',
 'Quest Hotel Darmo - Surabaya by ASTON - 0.08833161676427399',
 'Narita Classic Hotel Surabaya - 0.08833161676427399',
 'Premier Place Surabaya Airport - 0.08833161676427399',
 'Zoom Hotel Jemursari - 0.08833161676427399',
 'Cleo Hotel Basuki Rahmat - 0.0749975275102063',
 'Grand Darmo Suite by AMITHYA - 0.07174383261974006',
 'Varna Culture Hotel Surabaya - 0.06823993211613247']

## Improving Content-based Filtering 

In [11]:
# 1. Gunakan juga dataset review hotel
review_df = pd.read_excel('review_hotel.xlsx')
review_df.head()

,booking_id,booking_date,hotel_id,hotel_name,stay_duration,adults,children,rating,review
0,b0001,19-04-2020,h0014,Zest Hotel Jemursari Surabaya,1,2,1,8.4,Short stay
1,b0002,06-04-2020,h0014,Zest Hotel Jemursari Surabaya,1,1,1,10.0,Hotelnya nyaman
2,b0003,24-03-2020,h0014,Zest Hotel Jemursari Surabaya,2,2,1,9.2,Cukup baik untuk transit
3,b0004,23-03-2020,h0014,Zest Hotel Jemursari Surabaya,1,2,0,9.2,Nyaman
4,b0005,14-03-2020,h0014,Zest Hotel Jemursari Surabaya,2,2,1,6.8,Not good


In [12]:
# 2. Cek data null
review_df.isna().sum()

booking_id         0
booking_date       0
hotel_id         473
hotel_name         0
stay_duration      0
adults             0
children           0
rating             0
review             0
dtype: int64

In [13]:
# 3. Hapus data dimana hotel_id = null
# karena tanpa hotel_id maka data review tidak dapat direlasikan ke data hotel
review_df.dropna(inplace=True)

In [14]:
# 4. Group data review untuk setiap hotel
review_df = review_df.groupby('hotel_id', as_index=True).agg({'stay_duration':'mean', 'adults':'mean','children':'mean', 'rating':['mean','count']})

In [15]:
review_df.head()

stay_duration adults children rating      
                  mean   mean     mean   mean count
hotel_id                                           
h0001              1.4    1.6      0.4   9.20     5
h0002              1.3    1.3      0.5   8.32    10
h0003              1.6    1.6      0.4   8.76    10
h0004              1.3    1.5      0.5   8.52    10
h0005              1.4    1.6      0.3   8.04    10

In [16]:
hotel_df.head()

,hotel_id,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night,desc_clean
hotel_name,,,,,,,
favehotel Graha Agung Surabaya,h0002,Sebuah Hotel Mewah di Surabaya Persembahan dar...,Jawa Timur,Surabaya,Jl. Mayjen Yono Soewoyo Pakuwon Indah Square A...,442860.000000,buah hotel mewah surabaya sembah archipelago i...
The Sun Hotel Sidoarjo,h0003,Hotel Bintang 3 Pertama dan Satu-satunya di Si...,Jawa Timur,Surabaya,"Jl. Pahlawan No.1,Sidokumpul, Sidoarjo, Suraba...",305000.000000,hotel bintang 3 pertama satusatunya sidoarjo
Grand Surabaya Hotel,h0004,Penginapan Yang Tenang Dan Nyaman Di Surabaya.,Jawa Timur,Surabaya,"Jl. Pemuda 19-21, Surabaya, Indonesia",324999.333333,inap tenang nyaman surabaya
The WIN Hotel Surabaya,h0005,WIN Hotel adalah hotel smart bintang 3 yang me...,Jawa Timur,Surabaya,"Jl. Embong Tanjung 46 - 48 Surabaya, Jawa Timu...",310947.250000,win hotel hotel smart bintang 3 tawar layan tu...
POP! Hotel Stasiun Kota Surabaya,h0006,Sebuah Hotel Modern yang Terjangkau di Jawa Timur,Jawa Timur,Surabaya,"Jl. Waspada No.58, Surabaya, Jawa Timur, Indon...",276333.333333,buah hotel modern jangkau jawa timur


In [17]:
# 5. Gabungkan data hotel dan review
hotel_review_df = hotel_df.merge(review_df, how='inner', on='hotel_id', right_index=True)

/Users/ariflaksito/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [18]:
hotel_review_df.head()

,hotel_id,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night,desc_clean,"(stay_duration, mean)","(adults, mean)","(children, mean)","(rating, mean)","(rating, count)"
hotel_name,,,,,,,,,,,,
favehotel Graha Agung Surabaya,h0002,Sebuah Hotel Mewah di Surabaya Persembahan dar...,Jawa Timur,Surabaya,Jl. Mayjen Yono Soewoyo Pakuwon Indah Square A...,442860.000000,buah hotel mewah surabaya sembah archipelago i...,1.3,1.3,0.5,8.32,10
The Sun Hotel Sidoarjo,h0003,Hotel Bintang 3 Pertama dan Satu-satunya di Si...,Jawa Timur,Surabaya,"Jl. Pahlawan No.1,Sidokumpul, Sidoarjo, Suraba...",305000.000000,hotel bintang 3 pertama satusatunya sidoarjo,1.6,1.6,0.4,8.76,10
Grand Surabaya Hotel,h0004,Penginapan Yang Tenang Dan Nyaman Di Surabaya.,Jawa Timur,Surabaya,"Jl. Pemuda 19-21, Surabaya, Indonesia",324999.333333,inap tenang nyaman surabaya,1.3,1.5,0.5,8.52,10
The WIN Hotel Surabaya,h0005,WIN Hotel adalah hotel smart bintang 3 yang me...,Jawa Timur,Surabaya,"Jl. Embong Tanjung 46 - 48 Surabaya, Jawa Timu...",310947.250000,win hotel hotel smart bintang 3 tawar layan tu...,1.4,1.6,0.3,8.04,10
POP! Hotel Stasiun Kota Surabaya,h0006,Sebuah Hotel Modern yang Terjangkau di Jawa Timur,Jawa Timur,Surabaya,"Jl. Waspada No.58, Surabaya, Jawa Timur, Indon...",276333.333333,buah hotel modern jangkau jawa timur,1.4,1.2,0.6,7.52,10


In [19]:
hotel_review_df.columns

Index([               'hotel_id',       'hotel_description',
                'hotel_province',              'hotel_city',
                 'hotel_address',         'price_per_night',
                    'desc_clean', ('stay_duration', 'mean'),
              ('adults', 'mean'),      ('children', 'mean'),
              ('rating', 'mean'),       ('rating', 'count')],
      dtype='object')

In [20]:
# 6. Rename colums hasil merge
hotel_review_df.rename(columns={hotel_review_df.columns[7]:'stay_duration', hotel_review_df.columns[8]:'adults',
                               hotel_review_df.columns[9]:'children', hotel_review_df.columns[10]:'rating', 
                               hotel_review_df.columns[11]:'num_votes'}, inplace=True)

In [21]:
# 7. Membagi kategori hotel berdasar harga
hotel_review_df['price_per_night'] = hotel_review_df['price_per_night'].astype(int) 
hotel_review_df['price_bin'] = pd.cut(hotel_review_df['price_per_night'], 
                                      bins=3, labels=['low','medium','high'])

In [22]:
hotel_review_df.head()

,hotel_id,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night,desc_clean,stay_duration,adults,children,rating,num_votes,price_bin
hotel_name,,,,,,,,,,,,,
favehotel Graha Agung Surabaya,h0002,Sebuah Hotel Mewah di Surabaya Persembahan dar...,Jawa Timur,Surabaya,Jl. Mayjen Yono Soewoyo Pakuwon Indah Square A...,442860,buah hotel mewah surabaya sembah archipelago i...,1.3,1.3,0.5,8.32,10,low
The Sun Hotel Sidoarjo,h0003,Hotel Bintang 3 Pertama dan Satu-satunya di Si...,Jawa Timur,Surabaya,"Jl. Pahlawan No.1,Sidokumpul, Sidoarjo, Suraba...",305000,hotel bintang 3 pertama satusatunya sidoarjo,1.6,1.6,0.4,8.76,10,low
Grand Surabaya Hotel,h0004,Penginapan Yang Tenang Dan Nyaman Di Surabaya.,Jawa Timur,Surabaya,"Jl. Pemuda 19-21, Surabaya, Indonesia",324999,inap tenang nyaman surabaya,1.3,1.5,0.5,8.52,10,low
The WIN Hotel Surabaya,h0005,WIN Hotel adalah hotel smart bintang 3 yang me...,Jawa Timur,Surabaya,"Jl. Embong Tanjung 46 - 48 Surabaya, Jawa Timu...",310947,win hotel hotel smart bintang 3 tawar layan tu...,1.4,1.6,0.3,8.04,10,low
POP! Hotel Stasiun Kota Surabaya,h0006,Sebuah Hotel Modern yang Terjangkau di Jawa Timur,Jawa Timur,Surabaya,"Jl. Waspada No.58, Surabaya, Jawa Timur, Indon...",276333,buah hotel modern jangkau jawa timur,1.4,1.2,0.6,7.52,10,low


In [23]:
def set_family_room(x):
    if((x['children']>=1.0) | (x['adults']>1.0)):
        return "family room"
    else:
        return "single room"

In [24]:
# 8. Set column family_room/no untuk setiap item hotel
hotel_review_df['preferences'] = hotel_review_df.apply(lambda x : set_family_room(x), axis=1)

In [25]:
hotel_review_df.tail()

,hotel_id,hotel_description,hotel_province,hotel_city,hotel_address,price_per_night,desc_clean,stay_duration,adults,children,rating,num_votes,price_bin,preferences
hotel_name,,,,,,,,,,,,,,
De'kayakini Hotel,h0787,De'kayakini Hotel is located in the heart of t...,Jawa Barat,Bandung,"Cipamokolan, Rancasari, Bandung City, West Jav...",118438,dekayakini hotel is located in the heart of th...,3.0,1.0,0.0,8.00,2,low,single room
Best Western Premier La Grande,h0788,Hotel mewah bintang 4 dengan fasilitas dan des...,Jawa Barat,Bandung,"Jl. Merdeka No.25-29, Sumurbandung, Bandung, J...",433332,hotel mewah bintang 4 fasilitas desain baik ba...,2.0,1.7,0.7,9.12,20,low,family room
New Moonlight Hotel,h0789,Penginapan Yang Tenang Dan Nyaman Di Bandung.\...,Jawa Barat,Bandung,"Jl. Terusan Pasir koja No. 24-26, Bandung, Ind...",203628,inap tenang nyaman bandung tempat baik dekat p...,2.1,1.6,0.2,8.92,20,low,family room
Hotel Dafam Rio,h0790,Hotel yang menarik dan terjangkau persembahan ...,Jawa Barat,Bandung,"Jl. R.E. Martadinata No.160, Merdeka, Sumur Ba...",413451,hotel tarik jangkau sembah dafam group bandung...,1.7,1.6,0.8,9.08,20,low,family room
The Luxton Bandung,h0791,"Salah Satu Hotel Bintang 4 Terbaik di Dago, Ba...",Jawa Barat,Bandung,"Jl. Ir. H. Juanda No. 18, Citarum, Riau Street...",478400,salah satu hotel bintang 4 baik dago bandung l...,2.1,1.5,1.3,7.44,20,low,family room


In [26]:
# 9. Mix column hotel_province, hotel_city, desc_clean, price_bin dan preferences sebagai metadata soup
hotel_review_df['metadata'] = hotel_review_df[['hotel_province','hotel_city','desc_clean','price_bin','preferences']].agg(' '.join, axis=1)

In [27]:
# 10. Hitung TF-IDF dan Cosine similarity
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,)
tfidf_matrix = tf.fit_transform(hotel_review_df['metadata'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.12635074, 0.20350422, ..., 0.00908666, 0.02672682,
        0.00991107],
       [0.12635074, 1.        , 0.18484843, ..., 0.01072116, 0.02211741,
        0.01854756],
       [0.20350422, 0.18484843, 1.        , ..., 0.05519696, 0.01416575,
        0.00932333],
       ...,
       [0.00908666, 0.01072116, 0.05519696, ..., 1.        , 0.07137842,
        0.12399389],
       [0.02672682, 0.02211741, 0.01416575, ..., 0.07137842, 1.        ,
        0.08613599],
       [0.00991107, 0.01854756, 0.00932333, ..., 0.12399389, 0.08613599,
        1.        ]])

In [28]:
indices = pd.Series(hotel_review_df.index)
indices[:15]

0       favehotel Graha Agung Surabaya
1               The Sun Hotel Sidoarjo
2                 Grand Surabaya Hotel
3               The WIN Hotel Surabaya
4     POP! Hotel Stasiun Kota Surabaya
5                 Evora Hotel Surabaya
6         MaxOneHotels at Dharmahusada
7              Zoom Dharmahusada Hotel
8          Neo+ Waru Sidoarjo by ASTON
9                   favehotel Sidoarjo
10       News Front One Hotel Surabaya
11             Gunawangsa Manyar Hotel
12       Zest Hotel Jemursari Surabaya
13             The Life Hotel Surabaya
14          Best Western Papilio Hotel
Name: hotel_name, dtype: object

In [29]:
# 11. Modelling recommender system
def new_recommendations(name, top = 10):
    
    recommended_hotel = []
    
    idx = indices[indices == name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    top = top + 1
    top_indexes = list(score_series.iloc[0:top].index)
    
    for i in top_indexes:
        recommended_hotel.append(list(hotel_review_df.index)[i]+" - "+str(score_series[i]))
        
    return recommended_hotel

In [30]:
# Contoh menampilkan rekomendasi hotel
new_recommendations('Gunawangsa Manyar Hotel', 15)

['Gunawangsa Manyar Hotel - 1.0000000000000002',
 'Quest Hotel Darmo - Surabaya by ASTON - 0.42723671366986155',
 'POP! Hotel Stasiun Kota Surabaya - 0.33006159591777406',
 'Continent My Tower Hotel, Surabaya - Rungkut - 0.3040900608657291',
 'Grand Surabaya Hotel - 0.3040900608657291',
 'POP! Hotel Gubeng - 0.2996574810759386',
 'favehotel Graha Agung Surabaya - 0.2693913872971972',
 'Swiss-Belinn Tunjungan Surabaya - 0.266899783767704',
 'Zoom Dharmahusada Hotel - 0.26494804106486575',
 'Yello Hotel Jemursari - 0.24767689782258925',
 'Hotel Gunawangsa MERR - 0.24395207666894073',
 'The Life Hotel Surabaya - 0.24395207666894073',
 'News Front One Hotel Surabaya - 0.23341736389375126',
 'Zest Hotel Jemursari Surabaya - 0.2163636781397581',
 'Evora Hotel Surabaya - 0.18628828656848',
 'favehotel Sidoarjo - 0.1701541349297976']

In [31]:
hotel_review_df.loc['Gunawangsa Manyar Hotel']

hotel_id                                                         h0013
hotel_description    Sebuah Hotel bintang 3 yang Mudah, Simpel, dan...
hotel_province                                              Jawa Timur
hotel_city                                                    Surabaya
hotel_address                   Menur Pumpungan No 62 Manyar, Surabaya
price_per_night                                                 449619
desc_clean            buah hotel bintang 3 mudah simpel segar surabaya
stay_duration                                                      1.4
adults                                                             1.6
children                                                           0.4
rating                                                            7.84
num_votes                                                           10
price_bin                                                          low
preferences                                                family room
metada

In [33]:
hotel_review_df.loc['Quest Hotel Darmo - Surabaya by ASTON']

hotel_id                                                         h0022
hotel_description    Sebuah Hotel yang Nyaman, Lengkap, dan Terjang...
hotel_province                                              Jawa Timur
hotel_city                                                    Surabaya
hotel_address        Jl. Ronggolawe No. 27-29, Surabaya, Jawa Timur...
price_per_night                                                 360217
desc_clean                  buah hotel nyaman lengkap jangkau surabaya
stay_duration                                                      1.6
adults                                                             1.6
children                                                           0.4
rating                                                             8.6
num_votes                                                           10
price_bin                                                          low
preferences                                                family room
metada